## Install required package

In [0]:
!pip install langchain==0.0.354
!pip install openai==1.3.6
!pip install PyPDF2 
!pip install faiss-cpu==1.7.4
!pip install tiktoken
!pip install pypdf panel notebook
!pip install unstructured
!pip install "unstructured[pptx]"
!pip install --upgrade --quiet  "unstructured[all-docs]"
!pip install langchain-community
!pip install --upgrade pip
!pip install requests "urllib3<2" 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [0]:
#Restart Python for the installation to take place as suggested by Databricks
dbutils.library.restartPython()

## Import libraries and environment variables

In [0]:
import os
import openai
from pathlib import Path
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
#from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
#from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import panel as pn
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import  FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings

In [0]:
#Get the environment variables
os.environ["OPENAI_API_TYPE"] = "Azure"
os.environ["OPENAI_API_VERSION"] = dbutils.secrets.get("Secret-Scope", key = "azure-openai-api-version")
os.environ["OPENAI_API_BASE"] = dbutils.secrets.get("Secret-Scope", key = "azure-openai-api-base")
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("Secret-Scope", key = "open-api-key")
deployment_embed_name= dbutils.secrets.get("Secret-Scope", key = "azure-embedding-openai-deployment-name")
embed_model_name= dbutils.secrets.get("Secret-Scope", key = "azure-embedding-openai-model-name")
openai_api_base= dbutils.secrets.get("Secret-Scope", key = "azure-openai-api-base")

## Document Loading

In [0]:
#Directory loader way
from langchain_community.document_loaders.directory import DirectoryLoader
loader = DirectoryLoader('/dbfs/FileStore/ViiV',show_progress=True)

## Document Pre-processing/Chunking

In [0]:
embeddings =AzureOpenAIEmbeddings(deployment=deployment_embed_name, model=embed_model_name, openai_api_base=openai_api_base,openai_api_type="azure", chunk_size=1)
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
#engine is replacement of deployment as we move from langchain 0.0.200-->0.0.317

/local_disk0/.ephemeral_nfs/envs/pythonEnv-35df65e6-1d01-4856-8b4d-bbb8de7a9460/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-35df65e6-1d01-4856-8b4d-bbb8de7a9460/lib/python3.10/site-packages/langchain_community/embeddings/azure_openai.py:112: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from [REDACTED] to [REDACTED]/openai.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-35df

In [0]:
try:
  ppt = loader.load()
  texts = text_splitter.split_documents(ppt)
  docsearch =FAISS.from_documents(
  texts, embeddings
  )
except FileNotFoundError:
    print('File does not exist.')


  0%|          | 0/75 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 75/75 [03:01<00:00,  2.43s/it]


In [0]:
# #individual file way
# from langchain_community.document_loaders import UnstructuredFileLoader
# loader = UnstructuredFileLoader("/dbfs/FileStore/ViiV/ViiV_Apretude_Non_Adopter_Executive_Summary_vF.pptx", mode="elements")

In [0]:
# docs = loader.load()
# docs[:5]

## Creating Vectorstore

In [0]:
docsearch.save_local("/dbfs/FileStore/Rakesh/Vectorstore8")

In [0]:
pip list

Package                        Version
------------------------------ -------------
absl-py                        1.0.0
accelerate                     0.25.0
aiohttp                        3.9.1
aiosignal                      1.3.1
antlr4-python3-runtime         4.9.3
anyio                          3.5.0
appdirs                        1.4.4
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
astor                          0.8.1
asttokens                      2.0.5
astunparse                     1.6.3
async-timeout                  4.0.3
attrs                          22.1.0
audioread                      3.0.1
azure-core                     1.29.1
azure-cosmos                   4.3.1
azure-storage-blob             12.19.0
azure-storage-file-datalake    12.14.0
backcall                       0.2.0
backoff                        2.2.1
bcrypt                         3.2.0
beautifulsoup4                 4.11.1
black                          22.6.0
bleach           